In [1]:
!pip install Flask pyngrok line-bot-sdk requests openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.4/758.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


## 如果是在Google Colab上執行，請先執行以下程式碼

In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
openai_api_key = userdata.get('OPENAI_API_KEY')
port = 5000

In [3]:
import os
from pyngrok import ngrok

In [5]:
ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5000, name="linebot_tunnel")
print(tunnel.public_url)

https://e76e-34-170-167-23.ngrok-free.app


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)

from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    print("EVENT: ", event)
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                            TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/May/2024 04:55:11] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}
BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"510385751758995729","quoteToken":"EouTWMPl_Jy_xJZvpW9LQbFHKtvv7SYLcsf8e7QQ9iXQ2XQx2-Dy3txhrFxt61M3krlRFB8r3bqFGq99wANJVF1EE6BmFn190sJfiV7rgEOI6Qx-aELdTALv1Ep7N7znj0KxZyD10sr0X5mUhmgnIw","text":"HI"},"webhookEventId":"01HZ3XMEHQEK5YHVECW85P1N4N","deliveryContext":{"isRedelivery":false},"timestamp":1717044918332,"source":{"type":"user","userId":"Uc9d6d3d031222ca0e86cebfd151d295e"},"replyToken":"6e92388c104946c0a7b170510d20c9be","mode":"active"}]}
EVENT:  type='message' source=UserSource(type='user', user_id='Uc9d6d3d031222ca0e86cebfd151d295e') timestamp=1717044918332 mode=<EventMode.ACTIVE: 'active'> webhook_event_id='01HZ3XMEHQEK5YHVECW85P1N4N' delivery_context=DeliveryContext(is_redelivery=False) reply_token='6e92388c104946c0a7b170510d20c9be' message=TextMessageContent(type='text', id='510385751758995

INFO:werkzeug:127.0.0.1 - - [30/May/2024 04:55:19] "POST / HTTP/1.1" 200 -


In [ ]:
ok.disconnect(tunnel.public_url)